# Parameter Sweep and Scaling Demo
Demonstrates running small parameter sweeps over Δt, λ, and lattice size, computing scaling relations, visualizations, and JSONL logging.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from simulation.lattice_solver import run_lattice_solver
from simulation.entropy_balance import run_entropy_balance
from analysis.scaling_laws import fit_scaling
from utils.logging_utils import initialize_log, log_event

In [ ]:
# Parameters for sweep
dt_values = [0.01, 0.02]
lambda_values = [0.5, 1.0]
lattice_sizes = [16, 32]
n_steps = 50
log_file = 'parameter_sweep_demo.jsonl'
os.makedirs(os.path.dirname(log_file) or '.', exist_ok=True)
initialize_log(log_file)

In [ ]:
# Run sweep
for dt in dt_values:
    for lam in lambda_values:
        for lattice_size in lattice_sizes:
            Phi = np.random.randn(lattice_size, lattice_size)
            V = np.random.randn(lattice_size, lattice_size, 2)
            S = np.random.randn(lattice_size, lattice_size)
            Phi_final, V_final, S_final = run_lattice_solver(Phi, V, S, dt=dt, n_steps=n_steps)
            Sigma_dot_history, S_final_updated = run_entropy_balance(Phi_final, V_final, S_final, dt=dt, n_steps=n_steps)
            scaling_results = fit_scaling(Phi_final, S_final_updated, Sigma_dot_history)
            metrics = {
                'dt': dt,
                'lambda': lam,
                'lattice_size': lattice_size,
                'Sigma_dot_mean': float(np.mean(Sigma_dot_history)),
                'scaling_exponent': float(scaling_results['exponent'])
            }
            log_event(metrics, log_file)

In [ ]:
# Visualizations for last run
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.title('Phi Field (Last Sweep)')
plt.imshow(Phi_final, cmap='viridis')
plt.colorbar()
plt.subplot(1,2,2)
plt.title('S Field (Last Sweep)')
plt.imshow(S_final_updated, cmap='plasma')
plt.colorbar()
plt.show()
plt.figure()
plt.title('Entropy Production Over Time (Last Sweep)')
plt.plot(Sigma_dot_history)
plt.xlabel('Timestep')
plt.ylabel('Σ̇')
plt.show()

### Extension Ideas
- Expand grid to more dt, λ, lattice size values
- Compare scaling exponents across parameter combinations
- Integrate spectral coherence analysis per sweep
- Automate summary plots for all sweep results